**📊 Projeto ETL – Pipeline de Vendas**

Este notebook mostra um pipeline completo de ETL (Extração, Transformação e Carga) aplicado a dados de vendas de uma loja.

🔹 Objetivo: Demonstrar o fluxo de dados desde a importação do CSV até análises em um banco SQLite.
🔹 O que você verá aqui:

Extração dos dados de vendas 📥

Limpeza e transformação para análise 🔄

Criação de coluna valor_total 💰

Carga dos dados em banco SQLite 💾

Consultas SQL para gerar insights rápidos 🔍

🎯 Benefício: Aprender a construir um pipeline ETL simples e didático, pronto para portfólio de Engenharia de Dados!

**ETAPA 1** – Preparar o ambiente 🛠️

Vamos importar as ferramentas que vamos usar:

pandas para manipular os dados 🗂️

sqlite3 para criar e acessar o banco de dados 💾

os para gerenciar pastas e arquivos 📁

In [1]:
import pandas as pd
import sqlite3
import os

print("✅ Bibliotecas importadas")


✅ Bibliotecas importadas


**ETAPA 2** – Extração de dados 📥

Aqui realizamos a leitura do arquivo CSV contendo os dados de vendas. Essa é a etapa de extração, onde trazemos os dados para dentro do nosso ambiente de trabalho.

In [2]:
# Carregar CSV
caminho_arquivo = "vendas.csv"

if os.path.exists(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo)
    print("✅ CSV carregado com sucesso!")
    display(df.head())
else:
    print("⚠️ Arquivo CSV não encontrado. Faça o upload novamente.")


✅ CSV carregado com sucesso!


,Produto,Cidade,Vendedor,Quantidade,Preco_Unitario,Total_Venda,Data_Venda
0,Calça,Rio de Janeiro,Lucas,1,342.11,342.11,2025-09-13
1,Vestido,Rio de Janeiro,Paulo,7,263.39,1843.73,2025-02-23
2,Relógio,Rio de Janeiro,Patrícia,7,334.39,2340.73,2025-08-20
3,Jaqueta,Rio de Janeiro,Marcos,3,497.52,1492.56,2025-09-18
4,Calça,Belo Horizonte,Carla,8,271.71,2173.68,2025-02-01


**ETAPA 3 –** Transformação de dados 🔄

🔍**Tratar valores nulos**

Substituímos valores ausentes por 0 para evitar problemas em cálculos futuros.

In [4]:
# Tratando valores nulos

df_tratado = df.fillna(0)
print("✅ Valores nulos tratados")


✅ Valores nulos tratados


**Padronizar nomes das colunas ✏️**

Ajustamos os nomes das colunas para formato snake_case (tudo minúsculo, com _), garantindo consistência.

In [5]:
# Padronizando nomes de colunas

df_tratado.columns = df_tratado.columns.str.lower().str.replace(" ", "_")
print("✅ Nomes das colunas padronizados")
print(df_tratado.columns)


✅ Nomes das colunas padronizados
Index(['produto', 'cidade', 'vendedor', 'quantidade', 'preco_unitario',
       'total_venda', 'data_venda'],
      dtype='object')


**Converter datas 📅**

Convertendo a coluna de datas para o formato datetime, permitindo análises temporais corretas.

In [6]:
# Convertendo datas

if "data_venda" in df_tratado.columns:
    df_tratado["data_venda"] = pd.to_datetime(df_tratado["data_venda"], errors="coerce")
    print("✅ Coluna de data convertida")


✅ Coluna de data convertida


**Criar coluna valor_total 💰**

Calculamos o total de cada venda multiplicando quantidade pelo preco_unitario. Essa coluna será usada em análises de receita e vendas.

In [7]:
# Criando coluna valor total

if "preco_unitario" in df_tratado.columns and "quantidade" in df_tratado.columns:
    df_tratado["valor_total"] = df_tratado["preco_unitario"] * df_tratado["quantidade"]
    print("✅ Coluna 'valor_total' criada")
display(df_tratado.head())


✅ Coluna 'valor_total' criada


,produto,cidade,vendedor,quantidade,preco_unitario,total_venda,data_venda,valor_total
0,Calça,Rio de Janeiro,Lucas,1,342.11,342.11,2025-09-13,342.11
1,Vestido,Rio de Janeiro,Paulo,7,263.39,1843.73,2025-02-23,1843.73
2,Relógio,Rio de Janeiro,Patrícia,7,334.39,2340.73,2025-08-20,2340.73
3,Jaqueta,Rio de Janeiro,Marcos,3,497.52,1492.56,2025-09-18,1492.56
4,Calça,Belo Horizonte,Carla,8,271.71,2173.68,2025-02-01,2173.68


**Carga no banco SQLite 💾**

Nesta etapa, criamos a pasta database (caso não exista) e carregamos o DataFrame tratado no banco SQLite. Essa é a fase de carga (Load) do pipeline ETL.

In [8]:
# Criar pasta database se não existir
if not os.path.exists("database"):
    os.makedirs("database")

# Conectar/criar banco SQLite dentro da pasta database
conexao = sqlite3.connect("database/vendas.db")

# Carregar o DataFrame tratado no banco
df_tratado.to_sql("vendas", conexao, if_exists="replace", index=False)
print("✅ Banco SQLite criado e atualizado com a coluna 'valor_total'")


✅ Banco SQLite criado e atualizado com a coluna 'valor_total'


**Consultas de validação 🔍**

Executamos consultas SQL no banco para validar os dados carregados e gerar insights, como o total de vendas e a quantidade de vendas por cidade.

In [9]:
consulta = """
SELECT cidade,
       SUM(valor_total) AS total_vendas,
       COUNT(*) AS qtd_vendas
FROM vendas
GROUP BY cidade
ORDER BY total_vendas DESC
"""

resultado = pd.read_sql_query(consulta, conexao)
display(resultado)


,cidade,total_vendas,qtd_vendas
0,Fortaleza,27405.28,18
1,Rio de Janeiro,26238.26,17
2,Curitiba,25410.75,17
3,São Paulo,21958.43,11
4,Belo Horizonte,21299.86,13
5,Salvador,18435.59,16
6,Porto Alegre,13915.86,8


**Fechar conexão 🔒**

Após finalizar as consultas, encerramos a conexão com o banco de dados, garantindo que o arquivo SQLite seja salvo corretamente e evitando problemas de acesso.

In [10]:
conexao.close()
print("✅ Conexão com o banco encerrada")


✅ Conexão com o banco encerrada
